In [22]:
import numpy as np
import pandas as pan
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import classification_report as report

# We must first build a dataset consisting of predictions from Model 1 (CNN) and Model 2 (RBF-SVM):
# WARNING: The CNN csv file contains the labels so it MUST be append to the RHS of the DataFrame
df = pan.read_csv('svm_predictions.csv').join(pan.read_csv('cnn_predictions.csv'))
df = df.sample(frac=1).reset_index(drop=True)
df.to_csv('joint_dataframe.csv', index=False)
df

,desert,forest,grassland,tundra,desert_pred,forest_pred,grassland_pred,tundra_pred,desert_label,forest_label,grassland_label,tundra_label
0,0,0,1,0,0.648396,0.615052,0.992046,0.500000,0.0,0.0,1.0,0.0
1,1,0,0,0,0.996934,0.537442,0.635716,0.500000,1.0,0.0,0.0,0.0
2,0,1,0,0,0.788354,0.539963,0.935127,0.617764,1.0,0.0,0.0,0.0
3,1,0,0,0,0.994196,0.527019,0.686047,0.500000,1.0,0.0,0.0,0.0
4,0,0,0,1,0.628058,0.500000,0.500000,0.996709,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1111,0,0,1,0,0.989961,0.562256,0.694295,0.599990,1.0,0.0,0.0,0.0
1112,0,1,0,0,0.703726,0.500000,0.500000,0.991097,0.0,0.0,0.0,1.0
1113,0,1,0,0,0.579740,0.508693,0.995948,0.500000,0.0,0.0,1.0,0.0
1114,1,0,0,0,0.518181,0.522336,0.996445,0.560240,0.0,0.0,1.0,0.0


In [2]:
# Now we must split the data into train and test partitions:
train_in, test_in, train_out, test_out = tts(df.iloc[:,0:8], df.iloc[:,8:12], test_size=0.1)
print("Data Split!")

Data Split!


## **Perceptron Interpretation:**

In [14]:
# Now we must build a model that makes a join prediction; this interpretation will be a Perceptron:
joint_model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=8, activation='relu'),
    tf.keras.layers.Dense(units=4, activation='sigmoid')
])
joint_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),loss=tf.losses.CategoricalCrossentropy(from_logits=False),metrics=["accuracy"])
print("Model Built!")
print("GPU Available: ")
print(tf.config.list_physical_devices('GPU'))

Model Built!
GPU Available: 
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [15]:
joint_model.fit(
    x=train_in,
    y=train_out,
    verbose=0,
    epochs=70
)
print("Model Trained!")

Model Trained!


In [16]:
# Now we will report on the statistics for our model:
pred = joint_model.predict(test_in, verbose=0)
discrete_pred = []
for x in pred:
    index = np.argmax(x)
    if index == 0:
        discrete_pred.append([1,0,0,0])
    elif index == 1:
        discrete_pred.append([0,1,0,0])
    elif index == 2:
        discrete_pred.append([0,0,1,0])
    elif index == 3:
        discrete_pred.append([0,0,0,1])
stats = report(test_out, discrete_pred, target_names=['desert','forest','grassland','tundra'])
print("Report for Perceptron Interpretation of the Joint Model: \n" + stats)

Report for Perceptron Interpretation of the Joint Model: 
              precision    recall  f1-score   support

      desert       0.92      0.96      0.94        23
      forest       0.88      1.00      0.93        28
   grassland       0.95      0.87      0.91        23
      tundra       1.00      0.92      0.96        38

   micro avg       0.94      0.94      0.94       112
   macro avg       0.94      0.94      0.93       112
weighted avg       0.94      0.94      0.94       112
 samples avg       0.94      0.94      0.94       112



## **K-Fold CV Perceptron Interpretation:**

In [12]:
from sklearn.model_selection import KFold as kfd
from sklearn.metrics import accuracy_score as accuracy
from sklearn.metrics import mean_squared_error as mse

AVERAGE_MSE = 0
AVERAGE_ACC = 0
K_FOLDS = 10

# We must first split our joint DataFrame into 10 sets:
cv_train_frames = []
cross_val = kfd(n_splits=K_FOLDS)
for i, j in cross_val.split(df):
    cv_train_frames.append([pan.concat([df.iloc[0:j[0],:], df.iloc[(j[len(j)-1]+1):len(df),:]]), df.iloc[j[0]:(j[len(j)-1]+1),:]])

for k in range(K_FOLDS):
    instant_acc = 0
    instant_mse = 0
    joint_model = tf.keras.Sequential([
        tf.keras.layers.Dense(units=8, activation='relu'),
        tf.keras.layers.Dense(units=4, activation='sigmoid')
    ])
    joint_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),loss=tf.losses.CategoricalCrossentropy(from_logits=False),metrics=["accuracy"])
    joint_model.fit(
        x=cv_train_frames[k][0].iloc[:,0:8],
        y=cv_train_frames[k][0].iloc[:,8:12],
        verbose=0,
        epochs=70
    )
    pred = joint_model.predict(cv_train_frames[k][1].iloc[:,0:8], verbose=0)
    discrete_pred = []
    for x in pred:
        index = np.argmax(x)
        if index == 0:
            discrete_pred.append([1,0,0,0])
        elif index == 1:
            discrete_pred.append([0,1,0,0])
        elif index == 2:
            discrete_pred.append([0,0,1,0])
        elif index == 3:
            discrete_pred.append([0,0,0,1])
    instant_acc = accuracy(cv_train_frames[k][1].iloc[:,8:12], discrete_pred)
    instant_mse = mse(cv_train_frames[k][1].iloc[:,8:12], discrete_pred)
    AVERAGE_MSE += instant_mse
    AVERAGE_ACC += instant_acc
    print("Accuracy and MSE for iteration " + str(k+1) + ": " + str(round(instant_acc,3)) + " " + str(round(instant_mse,3)))
AVERAGE_MSE /= K_FOLDS
AVERAGE_ACC /= K_FOLDS
print("Average Accuracy and MSE for Perceptron Interpretation: " + str(round(AVERAGE_ACC,3)) + " " + str(round(AVERAGE_MSE,3)))

Accuracy and MSE for iteration 1: 0.929 0.036
Accuracy and MSE for iteration 2: 0.857 0.071
Accuracy and MSE for iteration 3: 0.839 0.08
Accuracy and MSE for iteration 4: 0.946 0.027
Accuracy and MSE for iteration 5: 0.955 0.022
Accuracy and MSE for iteration 6: 0.795 0.103
Accuracy and MSE for iteration 7: 0.856 0.072
Accuracy and MSE for iteration 8: 0.838 0.081
Accuracy and MSE for iteration 9: 0.874 0.063
Accuracy and MSE for iteration 10: 0.91 0.045
Average Accuracy and MSE for Perceptron Interpretation: 0.88 0.06


## **Export Model 3**

In [21]:
joint_model.save('.\\'+'saved_models'+'\\'+'model_3.h5')